In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('./Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [6]:
# Convert String values into numerical values
df2 = df.copy()
# df2 = pd.get_dummies(df, columns=['home_ownership','verification_status'])

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df2['home_ownership'] = le.fit_transform(df2['home_ownership']) 
df2['verification_status'] = le.fit_transform(df2['verification_status'])
df2['loan_status'] = le.fit_transform(df2['loan_status'])
df2['pymnt_plan'] = le.fit_transform(df2['pymnt_plan'])
df2['initial_list_status'] = le.fit_transform(df2['initial_list_status'])
df2['pymnt_plan'] = le.fit_transform(df2['pymnt_plan'])
df2['application_type'] = le.fit_transform(df2['application_type'])
df2['hardship_flag'] = le.fit_transform(df2['hardship_flag'])
df2['debt_settlement_flag'] = le.fit_transform(df2['debt_settlement_flag'])
df2['issue_d'] = le.fit_transform(df2['issue_d'])
df2['next_pymnt_d'] = le.fit_transform(df2['next_pymnt_d'])


# df2['issue_d'] = pd.to_datetime(df['issue_d'])
df2.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,2,1,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,1,105000.0,2,2,1,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,1,56000.0,2,2,1,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,3,92000.0,2,2,1,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,1,52000.0,0,2,1,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 86 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   68817 non-null  float64
 1   int_rate                    68817 non-null  float64
 2   installment                 68817 non-null  float64
 3   home_ownership              68817 non-null  int32  
 4   annual_inc                  68817 non-null  float64
 5   verification_status         68817 non-null  int32  
 6   issue_d                     68817 non-null  int32  
 7   loan_status                 68817 non-null  int32  
 8   pymnt_plan                  68817 non-null  int64  
 9   dti                         68817 non-null  float64
 10  delinq_2yrs                 68817 non-null  float64
 11  inq_last_6mths              68817 non-null  float64
 12  open_acc                    68817 non-null  float64
 13  pub_rec                     688

In [8]:
# Create our features
X = df2.copy()
# X = X.drop(columns="loan_status")
X = X.drop("loan_status", axis=1)

# Create our target
y = df2["loan_status"]
# y = df["loan_status"].values.reshape(-1, 1)

In [9]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.805542,0.0,21.778153,0.217766,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.714932,0.0,20.199244,0.718367,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.0,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.000000,0.0,13.890000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,1.000000,0.0,19.760000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,1.000000,0.0,26.660000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,2.000000,0.0,999.000000,18.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [10]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [11]:
y.describe()

count    68817.000000
mean         0.994958
std          0.070831
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: loan_status, dtype: float64

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
                                                    X, y, 
                                                    random_state=78, train_size=0.75)


In [13]:
print(Counter(y_train))
print(Counter(y_test))

Counter({1: 51357, 0: 255})
Counter({1: 17113, 0: 92})


# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [14]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

# brf_model = BalancedRandomForestClassifier(n_estimators=128, random_state=78) 
brf_model = BalancedRandomForestClassifier(n_estimators=128, random_state=1) 
brf_model

BalancedRandomForestClassifier(n_estimators=128, random_state=1)

In [15]:
# Fitting the model
brf_model = brf_model.fit(X_train, y_train)

In [16]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = brf_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7226834925901742

In [17]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[   52,    40],
       [ 2051, 15062]], dtype=int64)

In [18]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.02      0.57      0.88      0.05      0.71      0.48        92
          1       1.00      0.88      0.57      0.94      0.71      0.51     17113

avg / total       0.99      0.88      0.57      0.93      0.71      0.51     17205



In [19]:
importances = brf_model.feature_importances_
importances

array([0.01179774, 0.03167933, 0.01657461, 0.00264747, 0.01667988,
       0.00587574, 0.0234934 , 0.        , 0.01781637, 0.00419799,
       0.00934814, 0.00924518, 0.00073831, 0.01469273, 0.00985833,
       0.00123622, 0.0162097 , 0.01326368, 0.07154253, 0.04834475,
       0.07731957, 0.04661277, 0.00968506, 0.        , 0.        ,
       0.06913417, 0.00505505, 0.0002007 , 0.        , 0.00094629,
       0.        , 0.00343213, 0.01290102, 0.00511507, 0.00624894,
       0.00387568, 0.00747001, 0.01237432, 0.01320783, 0.01488644,
       0.00509643, 0.00531727, 0.01367532, 0.01480271, 0.01323515,
       0.00656608, 0.00567144, 0.00829077, 0.00976974, 0.01289717,
       0.0170576 , 0.01641166, 0.0002014 , 0.        , 0.0141528 ,
       0.01951962, 0.00969206, 0.01076733, 0.00633785, 0.00968375,
       0.01832621, 0.00283036, 0.00812197, 0.0091843 , 0.00878034,
       0.01024851, 0.01175412, 0.00920482, 0.01188814, 0.00866467,
       0.00961212, 0.        , 0.        , 0.00026387, 0.00675

In [28]:
# List the features sorted in descending order by feature importance
sorted(zip(brf_model.feature_importances_, X.columns), reverse=True)

[(0.07731956539730345, 'total_rec_prncp'),
 (0.07154253094311898, 'total_pymnt'),
 (0.06913416715004798, 'last_pymnt_amnt'),
 (0.048344753490800606, 'total_pymnt_inv'),
 (0.04661276958546753, 'total_rec_int'),
 (0.031679334018704625, 'int_rate'),
 (0.023493404041624817, 'issue_d'),
 (0.019519616400377155, 'mo_sin_old_rev_tl_op'),
 (0.018326207547048733, 'mths_since_recent_inq'),
 (0.017816370310375685, 'dti'),
 (0.0170576013906405, 'bc_open_to_buy'),
 (0.01667987971877563, 'annual_inc'),
 (0.016574610704077515, 'installment'),
 (0.016411661613370886, 'bc_util'),
 (0.016209695100933445, 'out_prncp'),
 (0.014886440109200628, 'il_util'),
 (0.014802714147034272, 'all_util'),
 (0.014692730236743768, 'revol_bal'),
 (0.014152803009415293, 'mo_sin_old_il_acct'),
 (0.013956669845529467, 'total_bc_limit'),
 (0.013739290249038277, 'tot_hi_cred_lim'),
 (0.013675315175888625, 'max_bal_bc'),
 (0.013263677197864594, 'out_prncp_inv'),
 (0.01323514937342451, 'total_rev_hi_lim'),
 (0.013207834688727112,

### Easy Ensemble AdaBoost Classifier

In [21]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [22]:
# print(X_train)
# print(y_train)
# ?EasyEnsembleClassifier

In [23]:
# Fit the model
eec.fit(X_train, y_train)

# # Make Prediction
predictions = eec.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).head(20)

,Prediction,Actual
3297,1,1
47738,1,1
64369,1,1
21868,1,1
67187,1,1
14175,1,1
30313,1,1
9849,1,1
31916,1,1
29415,1,1


In [24]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.7226834925901742

In [25]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
# y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   52,    40],
       [ 2051, 15062]], dtype=int64)

In [26]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.02      0.57      0.88      0.05      0.71      0.48        92
          1       1.00      0.88      0.57      0.94      0.71      0.51     17113

avg / total       0.99      0.88      0.57      0.93      0.71      0.51     17205

